In [1]:
!pip install pickle5

In [2]:
import pickle
import os
import numpy as np
from keras.layers import LSTM, Input, Dense, Activation, Embedding, Concatenate, Reshape
from keras.layers import RepeatVector, Permute
from keras.layers import Multiply, Lambda
import keras.backend as K 
from keras.models import Model
import tensorflow as tf
from keras.optimizers import RMSprop

2023-05-29 06:12:13.698276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarni

In [3]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pickle.load(input_file)

notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]
bars_to_int = dictionary["bars_to_int"]
int_to_bars = dictionary["int_to_bars"]

n_notes = len(notes_to_int)
n_phrases = len(phrases_to_int)
n_bars = len(bars_to_int)

In [4]:
def create_network(n_notes, n_bars, n_phrases, embed_size = 100, rnn_units = 256, use_attention = False):

    notes_in = Input(shape = (None,))
    bars_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x3 = Embedding(n_bars, embed_size)(bars_in)

    
    x = Concatenate()([x1,x3])

    x = LSTM(rnn_units, return_sequences=True)(x)

    if use_attention:

        x = LSTM(rnn_units, return_sequences=True)(x)

        e = Dense(1, activation='tanh')(x)
        e = Reshape([-1])(e)
        alpha = Activation('softmax')(e)

        alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

        c = Multiply()([x, alpha_repeated])
        c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
    
    else:
        c = LSTM(rnn_units)(x)
                                    
    phrases_out = Dense(n_phrases, activation = 'softmax', name = 'phrase')(c)
   
    model = Model([notes_in, bars_in], [phrases_out])
    

    if use_attention:
        att_model = Model([notes_in, bars_in], alpha)
    else:
        att_model = None


    opti = RMSprop(lr = 0.001)
    model.compile(loss=['binary_crossentropy', 'binary_crossentropy'], optimizer=opti)

    return model, att_model

In [5]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 42
rnn_units = 256
use_attention = True

model, att_model = create_network(n_notes, n_bars, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

2023-05-29 06:12:16.549816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 06:12:16.551989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 06:12:16.553449: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 42)     1764        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 42)     84          ['input_2[0][0]']                
                                                                                              

2023-05-29 06:12:16.808184: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 06:12:16.810112: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 06:12:16.811649: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [2]:
pieces_train = None
with open("../data/Oskar Kolberg's Dataset/train/pieces_train.pkl", 'rb') as handle:
    pieces_train = pickle.load(handle)

pieces_test = None
with open("../data/Oskar Kolberg's Dataset/test/pieces_test.pkl", 'rb') as handle:
    pieces_test = pickle.load(handle)

In [7]:
def get_boundaries(piece):
    notes_input = [ notes_to_int[note] for note in piece[0] ]
    bars_input = [ bars_to_int[bar] for bar in piece[2] ]
    expected = phrases_input = piece[3]
    
    max_extra_notes = len(notes_input)
    seq_len = 36
    phrases_output = []

    notes_input_sequence = [notes_to_int[129] for i in range(seq_len)] #[notes_input[0]] # the first note element of the sequence
    bars_input_sequence = [0 for i in range(seq_len)] #[bars_input[0]] # the first duration element of the sequence
    
#     notes_data = [129 for i in range(79)]
# durations_data = [0 for i in range(79)]
# bars_data = [0 for i in range(79)]
# phrases_data = [0 for i in range(79)]

    if seq_len is not None:
        notes_input_sequence = [notes_to_int[129]] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
        bars_input_sequence =  [0] * (seq_len - len(bars_input_sequence)) + bars_input_sequence # the same for durations

    for note_index in range(max_extra_notes):
        prediction_input = [
            np.array([notes_input_sequence]),
            np.array([bars_input_sequence])
        ]

        phrases_prediction = model.predict(prediction_input, verbose=0)
        phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

        notes_input_sequence.append(notes_input[note_index])
        bars_input_sequence.append(bars_input[note_index])
        
        if len(notes_input_sequence) > seq_len:
            notes_input_sequence = notes_input_sequence[1:]
            bars_input_sequence = bars_input_sequence[1:]
    actual = phrases_output
    
    return (actual, expected)

### Calculate F1 score

In [8]:
def f1_score(expected, actual):
    """
    Calculates the F1 score given two arrays of 0s and 1s (expected and actual).
    """
    # Ensure that the inputs are numpy arrays.
    expected = np.array(expected)
    actual = np.array(actual)
    
    # Calculate the true positives, false positives, and false negatives.
    tp = np.sum((expected == 1) & (actual == 1))
    fp = np.sum((expected == 0) & (actual == 1))
    fn = np.sum((expected == 1) & (actual == 0))
    
    # Calculate the precision, recall, and F1 score.
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    
    return f1

In [9]:
a, e = get_boundaries(pieces_test[99])
print(a)
print(e)

2023-05-29 06:12:17.358615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-29 06:12:17.360673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-29 06:12:17.362103: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


F1 for the train data

In [10]:
actual = []
expected = []
counter = 1 
correct_predictions = 1
for piece in pieces_train[:100]:
    correct = True
    print(counter, end= " ")
    a, e = get_boundaries(piece)
    actual.extend(a)
    expected.extend(e)
    for i in range(len(a)):
        if a[i] != e[i]:
            correct = False
    if correct:
        correct_predictions += 1
    counter += 1
    
f1_score(expected,actual)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

0.680952380952381

In [11]:
correct_predictions

33

F1 for the test data

In [12]:
actual = []
expected = []
counter = 1 
correct_predictions = 1
for piece in pieces_test[:100]:
    correct = True
    print(counter, end= " ")
    a, e = get_boundaries(piece)
    actual.extend(a)
    expected.extend(e)
    for i in range(len(a)):
        if a[i] != e[i]:
            correct = False
    if correct:
        correct_predictions += 1
    counter += 1
f1_score(expected,actual)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

0.6806526806526807

In [13]:
correct_predictions

35